# 데이터수집

## 크롤링을 이용한 서울시 스타벅스 매장 목록 데이터 생성

In [56]:
# 라이브러리 임포트

from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# webdriver 실행 후 스타벅스의 지역별 매장 검색 화면에 접속
driver = webdriver.Safari()
url = 'https://www.istarbucks.co.kr/store/store_map.do?disp=locale'
driver.get(url)

In [4]:
# webdriver로 '서울' 버튼 요소를 찾아 클릭
seoul_btn = '#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a'
driver.find_element_by_css_selector(seoul_btn).click()

/var/folders/xx/_99p48vj7pg7bs_kpl7838_00000gn/T/ipykernel_68416/262176323.py:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector(seoul_btn).click()


In [5]:
# webdriver로 '전체' 버튼 요소를 찾아 클릭

all_btn = '#mCSB_2_container > ul > li:nth-child(1) > a'
driver.find_element_by_css_selector(all_btn).click()

/var/folders/xx/_99p48vj7pg7bs_kpl7838_00000gn/T/ipykernel_68416/3377554648.py:4: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector(all_btn).click()


In [6]:
# BeautifulSoup으로 HTML 파서 만들기
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [70]:
# select()을 이용해 원하는 HTML 태그를 찾아오기
starbucks_soup_list = soup.select('li.quickResultLstCon')

len(starbucks_soup_list)

568

In [67]:
# 태그 문자열 살펴보기
starbucks_soup_list[0]

<li class="quickResultLstCon" data-code="3762" data-hlytag="null" data-index="0" data-lat="37.501087" data-long="127.043069" data-name="역삼아레나빌딩" data-storecd="1509" style="background:#fff"> <strong data-my_siren_order_store_yn="N" data-name="역삼아레나빌딩" data-store="1509" data-yn="N">역삼아레나빌딩  </strong> <p class="result_details">서울특별시 강남구 언주로 425 (역삼동)<br/>1522-3232</p> <i class="pin_general">리저브 매장 2번</i></li>

name   매장명
lat    위도
lng    적도
store_type   매장 타입
address  주소
tel    전화번호

In [63]:
# 스타벅스 매장 정보 샘플 확인
startbucks_store = starbucks_soup_list[1]
name = startbucks_store.select('strong')[0].text.strip()
lat = startbucks_store['data-lat'].strip()
lng = startbucks_store['data-long'].strip()
store_type = startbucks_store.select('i')[0]['class'][0][4:]  ##??
address1 = startbucks_store.select('p')[0]
address = str(startbucks_store.select('p')[0]).split('<br/>')[0].split('>')[1]
tel = str(startbucks_store.select('p')[0]).split('<br/>')[1].split('<')[0]

print(type(address1))
print(address)
print()
tel

<class 'bs4.element.Tag'>
서울특별시 강남구 강남대로 538 (논현동)



'1522-3232'

In [73]:
# 서울시 스타벅스 매장 목록 데이터 만들기

starbucks_list = []
for item in starbucks_soup_list:
    name = item.select('strong')[0].text.strip()
    lat = item['data-lat'].strip()
    lng = item['data-long'].strip()
    store_type = item.select('i')[0]['class'][0][4:] 
    address1 = item.select('p')[0]
    address = str(item.select('p')[0]).split('<br/>')[0].split('>')[1]
    tel = str(item.select('p')[0]).split('<br/>')[1].split('<')[0]

    starbucks_list.append([ name, lat, lng, store_type, address, tel])

In [74]:
# pandas의 데이터프레임 생성
columns = ['매장명','위도','경도','매장타입','주소','전화번호']
seoul_starbucks_df = pd.DataFrame(starbucks_list, columns = columns)
seoul_starbucks_df.head()

,매장명,위도,경도,매장타입,주소,전화번호
0,역삼아레나빌딩,37.501087,127.043069,general,서울특별시 강남구 언주로 425 (역삼동),1522-3232
1,논현역사거리,37.510178,127.022223,general,서울특별시 강남구 강남대로 538 (논현동),1522-3232
2,신사역성일빌딩,37.514132,127.020563,general,서울특별시 강남구 강남대로 584 (논현동),1522-3232
3,국기원사거리,37.499517,127.031495,general,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232
4,스탈릿대치R,37.494668,127.062583,reserve,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232


In [75]:
seoul_starbucks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568 entries, 0 to 567
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   매장명     568 non-null    object
 1   위도      568 non-null    object
 2   경도      568 non-null    object
 3   매장타입    568 non-null    object
 4   주소      568 non-null    object
 5   전화번호    568 non-null    object
dtypes: object(6)
memory usage: 26.8+ KB


In [76]:
seoul_starbucks_df.to_excel('./seoul_starbucks_list.xlsx',index=False)